# Import Libraries

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Import Data

In [11]:
data = pd.read_excel('Dataset.xlsx') 

# Check Data

In [12]:
print("Missing Data Summary:")
print(data.isnull().sum())

Missing Data Summary:
Team         0
Match Up     0
Game Date    0
W/L          0
MIN          0
PTS          0
FGM          0
FGA          0
FG%          0
3PM          0
3PA          0
3P%          0
FTM          0
FTA          0
FT%          0
OREB         0
DREB         0
REB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
+/-          0
dtype: int64


In [13]:
non_numeric_values =data. applymap(lambda x: isinstance(x, str) and not x.isnumeric()).any()
print("Columns with non-numeric values:")
print(non_numeric_values[non_numeric_values].index.tolist())

Columns with non-numeric values:
['Team', 'Match Up', 'Game Date', 'W/L', 'FT%']


/var/folders/s8/_l25g4l10y7725mc4fmr9mpm0000gn/T/ipykernel_19161/852359997.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_numeric_values =data. applymap(lambda x: isinstance(x, str) and not x.isnumeric()).any()


In [14]:
data['FT%'] = data['FT%'].replace('-', pd.NA)
data['FT%'] = pd.to_numeric(data['FT%'], errors='coerce')
data['FT%'].fillna(data['FT%'].mean(), inplace=True)

# Model

## Construct new data frame to for model construction
- get rid of '+/-'
- 'Home_Game'
- 'W/L'



In [15]:
df = data[['Team', 'Match Up', 'Game Date', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'STL', 'BLK', 'TOV', 'PF']]
df['Home_Game'] = df['Match Up'].apply(lambda x: 1 if "vs." in x else 0)
df = df.drop(columns = 'Match Up')
df['Game Date'] = pd.to_datetime(df['Game Date'], format='%m/%d/%Y')
df = df.sort_values(by=['Team', 'Game Date']).reset_index(drop=True)
df

,Team,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,Home_Game
0,ATL,2023-10-25,L,240,110,39,93,41.9,5,29,...,81.8,12,30,42,24,12,1,12,19,0
1,ATL,2023-10-27,L,240,120,42,87,48.3,12,32,...,80.0,9,35,44,28,7,6,14,20,1
2,ATL,2023-10-29,W,240,127,47,93,50.5,15,37,...,81.8,13,33,46,32,15,2,17,17,0
3,ATL,2023-10-30,W,240,127,48,86,55.8,14,30,...,94.4,4,32,36,28,6,7,11,12,1
4,ATL,2023-11-01,W,240,130,46,92,50.0,9,32,...,90.6,14,43,57,26,8,3,21,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,WAS,2024-04-05,L,240,102,38,88,43.2,5,33,...,70.0,12,33,45,24,9,5,10,21,1
2456,WAS,2024-04-07,L,240,122,42,89,47.2,16,40,...,73.3,8,35,43,29,9,11,15,25,0
2457,WAS,2024-04-09,L,240,121,44,89,49.4,20,43,...,76.5,7,34,41,26,5,4,13,22,0
2458,WAS,2024-04-12,L,240,127,45,86,52.3,18,43,...,65.5,4,35,39,32,6,5,12,22,1


## Splitting Data
- Each team will have 70% training data and 30% testing data

In [1]:
from utils import splitting_data
# X_train, y_train, X_test, y_test = splitting_data(df)

## Standardize data set

In [ ]:
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)


## Logistic Regression + LASSO

In [ ]:
from utils import lasso_regression
model, y_pred = lasso_regression(X_train, y_train, X_test)


ImportError: cannot import name 'lasso_regression' from 'utils' (/Users/yuertang/Desktop/2024 Fall/Stats 101C/Final_Projects/Stats101CProjects/utils.py)

## Weighting Statistics

In [ ]:
# Ensure 'Game Date' is in datetime format
df_model['Game Date'] = pd.to_datetime(df_model['Game Date'])

# Calculate 'days_ago' and 'weight'
reference_date = df_model['Game Date'].max()
df_model['days_ago'] = (reference_date - df_model['Game Date']).dt.days
df_model['weight'] = 1 / (1 + k * df_model['days_ago'])  # Create weights here

# Now split the data
train_data_list = []
test_data_list = []
df_model = df_model.sort_values(by='Team')  # Sort by team and game date
for team, group in df_model.groupby('Team'):
    train_size = int(len(group) * 0.7)
    train_data_list.append(group.iloc[:train_size])
    test_data_list.append(group.iloc[train_size:])

# Concatenate the training and testing data for all teams
train_data = pd.concat(train_data_list)
test_data = pd.concat(test_data_list)

# Access weights
sample_weights = train_data['weight']


Accuracy: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       370
           1       0.85      0.87      0.86       380

    accuracy                           0.85       750
   macro avg       0.86      0.85      0.85       750
weighted avg       0.85      0.85      0.85       750


Important Features:
      Feature  Coefficient
0   Home_Game    -0.071743
1         MIN    -0.007383
2         PTS     1.705375
3         FGM     2.001316
4         FGA    -3.556771
5         FG%    -0.755286
6         3PM    -0.268279
7         3PA     0.371768
8         3P%     0.518743
9         FTM     0.168712
10        FTA    -0.587519
11        FT%     0.118982
12       OREB     0.598383
13       DREB     1.252217
14        REB     1.420059
15        AST     0.138601
16        STL     1.403231
17        BLK     0.439733
18        TOV    -1.351008
19         PF    -0.369412
